__[A Baby Robot's Guide To Reinforcement Learning](https://towardsdatascience.com/tagged/baby-robot-guide)__

# Policy and Value Iteration
## An Introduction to Reinforcement Learning: Part 3

<center><img src="images/part3/value_iteration_header_optimized.gif"/></center>

# Introduction
Baby Robot has been steadily making his way back to his mum. So far he has:

* Overcome the Bandit Problem to recharge his batteries

* Solved the Prediction Problem to evaluate the relative value of states in an environment, letting him escape from a simple grid level.

* Seen how an environment can be represented as a Markov Decision Process (MDP) and evaluated using the Bellman Equations.


In this next instalment we'll consider the Control Problem and investigate how the optimal policy can be found from estimates of the environment's state values.

___

This notebook accompanies the Towards Data Science article and is part of _[A Baby Robot's Guide To Reinforcement Learning](https://towardsdatascience.com/tagged/baby-robot-guide)_

___

![](images/green_babyrobot_small.gif)

# Code
A note on the code used in this notebook
To keep this notebook relatively tidy, all of the main functionality is contained in Python classes, each of which is in its own file in the "lib" directory.

The images in this notebook are created using the fantastic _[ipycanvas](https://ipycanvas.readthedocs.io/en/latest/)_ library, which allows drawing to the browser canvas. Unfortunately these images aren't displayed by github's preview, so to view the images and play with the code you'll need to clone the github repository.

In [1]:
!pip install -U ipycanvas -q

In [2]:
import sys
sys.path.insert(0, './lib')

import os
import numpy as np
import random
from time import sleep, time
from enum import IntEnum

from ipywidgets import Layout
from ipywidgets import Play, IntProgress, HBox, VBox, link

from direction import Direction
from arrows import Arrows
from grid_level import GridLevel, setup_play_level
from robot_position import RobotPosition
from policy import Policy
from policy_evaluation import PolicyEvaluation

# Simple Grid Level
Baby Robot has found himself in a very small room, so escaping from this shouldn't be too much of a challenge. The only downside is that the roof has been leaking and, as we saw in the previous part when we looked at MDPs, Baby Robot doesn't like the wet. Puddles slow him down and potentially cause him to skid.

In [3]:
level_width = 3
level_height = 3
level = GridLevel( level_width,level_height,start=[0,0],end=[2,2],add_compass = True)
robot_position = RobotPosition(level,initial_sprite = 2)

splashes = [[0,2,1],
            [0,2,1],
            [0,1,0]]
level.add_splashes( splashes )

# add some walls
walls = [((0, 0),'E'),((1, 2),'E')]
level.add_walls( walls )
level.canvases

MultiCanvas(height=196, sync_image_data=True, width=296)

<center><i>Figure 1: The simple grid level. Puddles take longer to move through and potentially can cause a skid.</i></center>

We defined that:

### Dry Squares:
* Take one unit of time to move through, so a reward of -1 is received when moving to a square with no puddle.
* Baby Robot won't skid if there's no puddle, so the target state will always be reached.

### Small Puddles:
* Take twice as long to move through, so stepping in one of these gives a reward of -2.
* The probability of skidding when in a small puddle is 0.4, so 0.6 of the time you'll reach the target state, the rest of the time you'll skid and end up in one of the other possible states.

### Large Puddles:
* Take 4 times as long to move through, so a reward of -4 is given when entering a square containing a large puddle.
* There's an increased chance of skidding, so you'll only reach the target with a 0.4 probability. Therefore there's a 0.6 probability you'll end up in one of the other possible states.


This makes the problem slightly more complex, since it's no longer just a case of choosing which state you'd like to move to and getting a fixed reward. Now, when an action is chosen, Baby Robot may end up moving to a state that wasn't the target state and instead receives the reward for the state where he ends up.

<center><img src="images/part3/simple_level_move_east.png"/></center>
<br/>
<center><i>Figure 2: Baby Robot would like to move East towards the exit, but is in a large puddle so there's a chance of skidding to a different state. The transition probabilities, of moving to each of the possible next states, is shown in blue. The reward received for each transition is given in red.</i></center>

When a skid does occur the resultant state is decided by dividing the total probability of skidding by the number of possible non-target states. So, for the example shown in Figure 2 above, where Baby Robot is currently sitting in a large puddle and wanting to move East, towards the exit, the following next states and rewards are possible:

<center><img src="images/part3/table1_transition_probabilities.png"/></center>
<br/>
<center><i>Table 1: Transition probabilities for figure 2 - starting in the center square and wanting to move East.</i></center>

# Policy Evaluation
For the simple grid level, since we have complete information about the system, knowing all of its transition probabilities and rewards, we can use Dynamic Programming to turn the Bellman Equations into update functions. These can then be applied iteratively to calculate the state values.

The Bellman Expectation equation, giving the value of state 's' when following policy 'π' is given by:

<center><img src="images/part3/bellman_expectation_equation.png"/></center>
<br/>
<center><i>Equation 1: Bellman expectation equation giving the state value under policy π</i></center>


where:

* π(a|s) is the probability of taking action a in state s.

* p(s',r|s,a) is the probability of moving to the next state s' and getting reward r when starting in state s and taking action a.

* r is the reward received after taking this action.

* γ is the discount factor.

* v(s') is the value of the next state.


<blockquote>
<i>
The value of a state is given by the sum over the probability of all actions, times the sum over the probability of the next state and reward, multiplied by the reward obtained and the discounted value of the state where you end up.
</i>
</blockquote>

And as an update equation this can be written:


<br/>
<center><img src="images/part3/bellman_expectation_equation_update.png"/></center>
<br/>
<center><i>Equation 2: The Bellman expectation equation expressed as an update function.</i></center>
<br/>


The only difference here is that the value of the state 's', at iteration 'k+1', is calculated using the values of the next states that were already calculated at the previous iteration 'k'. So the new estimate of a state's value is derived from the previous estimates. When one estimate is based on another this is referred to as <b>Bootstrapping</b>.

In the example shown in Figure 2 above, where Baby Robot would like to choose the action to move East from the center square, we're using a deterministic policy. In other words, for each state our policy will explicitly define which action should be chosen. Therefore there'll be a single action, chosen with probability 1, so the sum over the actions in the above equations will disappear. 

Similarly, when using iterative policy evaluation, it's common to start with all the initial state value estimates set to zero, so the discounted value of the next states will also disappear for the first iteration. 

As a result, for the state at the center of the grid level, where Baby Robot currently finds himself, the first estimate of the state's value can be given by summing over the probabilities and rewards that are shown in the table above. So, for a policy that selects the East action in this state, the first estimate of the value of this state is:

<br/>
<center><img src="images/part3/v_e.png"/></center>

If, instead of selecting the East action in this state, the policy defined that actions of North, South or West should be taken, then the first estimates of the state value would be given by:

<br/>
<center><img src="images/part3/v_nws.png"/></center>

From these first state estimates, where no future rewards are considered (since all next state values are currently zero), a policy that chooses to move West has the least negative state value estimate. This is due to its target state being a dry square, which gives the least negative immediate reward from the possible states. This estimate doesn't yet contain any information about future rewards, which are required to let us find the best path through the grid. At the minute the estimated state value only the gives information that will keep Baby Robot the driest!

The state value estimates shown above are the values calculated for a deterministic policy, where a set action is defined for a state. If we instead used a stochastic policy, where actions are chosen probabilistically, then each of these state value estimates would simply be multiplied by the probability of that action being taken  (the π(a|s) part in the Bellman equation) and then we'd add the results. 

For the center square, with an equal probability of selecting any action (so each has a 0.25 probability of being chosen), the first estimated state value would be:

<br/>
<center><img src="images/part3/v_all.png"/></center>

In all cases this estimated value is a measure of how good it is to be in this state under the current policy. To find which are the best states to be in, which in turn will let us find the best route through the grid, we need to repeat our calculation for all other states in the grid (commonly referred to as doing a sweep of all states). For the stochastic policy, where all possible actions in a state are equally likely, this gives us the state value estimates shown below:

In [4]:
level = GridLevel( level_width,level_height,start=[0,0],end=[2,2],show_end_text = False)
level.add_splashes( splashes )
level.add_walls( walls )

# the possible directions
directions = level.get_available_directions()

# initially policy evaluation starts with all estimates set to zero
# and all available actions are equally likely
policy_evaluation = PolicyEvaluation( level )
policy = Policy(level)

# directions = policy.get_directions(policy_evaluation.end_values)
level.show_directions(directions)
level.show_values(policy_evaluation.end_values)
level.canvases

MultiCanvas(height=196, sync_image_data=True, width=196)

In [5]:
# run one step of evaluation
policy_evaluation.do_iteration()
level.show_values(policy_evaluation.end_values)

<center><i>Figure 3: State value estimates after the first iteration of policy evaluation for a stochastic policy, where all possible actions (shown by the blue arrows) are equally likely. (Note: the values have been rounded to 1 decimal place).</i></center>

Now that we have initial estimates for all states, we can use these as the next state values, <i>v(s')</i>, in the next iteration of our Bellman update equation. At each iteration of policy evaluation the policy, transition probabilities and rewards for moving to each state remain fixed, therefore it will only be the estimated next state values that change.

Consequently at the next iteration, with a discount factor of 1, the estimated state values for each of the possible actions for the center square become:

<br/>
<center><img src="images/part3/v_nsew.png"/></center>

And for the stochastic policy, where each action is equally likely, the state value is:

<br/>
<center><img src="images/part3/v_stochastic.png"/></center>

Repeating this for all states gives us our second set of estimates and this time the estimated state values do take the next state into account:

In [6]:
# run one step of evaluation
policy_evaluation.do_iteration()
level.show_values(policy_evaluation.end_values)
level.canvases

MultiCanvas(height=196, sync_image_data=True, width=196)

<center><i>Figure 4: State value estimates after the second iteration of policy evaluation for a stochastic policy, where all possible actions (shown by the blue arrows) are equally likely. (Note: the values have been rounded to 1 decimal place).</i></center>

If we repeat this process and run to convergence (or at least until the changes in the state values fall below some small threshold value), then we end up with the final state values shown in Figure 5, below. In this case convergence occurred after 234 iterations of Policy Evaluation.

In [7]:
# run until the change in estimated state values drops below the defined threshold
iterations = policy_evaluation.run_to_convergence(max_iterations = 300)
level.show_values(policy_evaluation.end_values)
print(f"Convergence in {iterations} iterations")
level.canvases

Convergence in 234 iterations


MultiCanvas(height=196, sync_image_data=True, width=196)

<center><i>Figure 5: Final state value estimates after convergence of policy evaluation for a stochastic policy, where all possible actions are equally likely. Convergence occurred after 234 iterations for a threshold value of 0.001.</i></center>

Finally, if we now act greedily with respect to these values, choosing the action in each state that moves to the highest valued (least negative) next state, we get the following policy:

In [8]:
# act greedily wrt the calculated state values
directions = policy.get_directions(policy_evaluation.end_values)
level.show_directions(directions)
level.show_values(policy_evaluation.end_values)
level.canvases

MultiCanvas(height=196, sync_image_data=True, width=196)

<center><i>Figure 6: The optimal policy, formed by acting greedily with respect to the estimated state values.</i></center>

For this very simple level, this policy, formed from acting greedily with respect to the state values after one pass of policy evaluation, is the optimal policy. For any state, if Baby Robot takes the action specified by this policy and then continues to follow this optimal policy, his expected return will be higher than following actions given by any other policy.


<blockquote>
<i>
For a finite Markov Decision Process (MDP), where we have full information about the state transitions and rewards, we can use Dynamic Programming to turn the Bellman Equations into a set of update rules. These can then be used to calculate the state values under the current policy.
</i>
</blockquote>

# Generalised Policy Iteration (GPI)
For the very simple grid level we considered above, using an initial stochastic policy, we were able to find the optimal policy by running a single iteration of policy evaluation and then improving the policy by acting greedily with respect to the calculated state values. 

In practice this isn't always possible. Instead policy evaluation and policy improvement need to be run multiple times - a technique that is labelled <b>Generalised Policy Iteration (GPI)</b>.

Under GPI, Policy Evaluation is used to calculate the state values for the current policy. Then Policy Improvement is applied, by acting greedily with respect to these values. This improves the current policy and creates the policy for the next iteration. By repeatedly applying this cycle of policy evaluation and improvement the policy is moved ever closer towards the optimal policy, as shown below:

<br/>
<center><img src="images/part3/gpi.png"/></center>

<center><i>Figure 7: Generalised Policy Iteration - repeatedly applying evaluation and improvement to move towards the optimal policy.</i></center>

At each step of Generalised Policy Iteration, the current policy will be evaluated and the policy improved. After each policy improvement the previously calculated state values are no longer valid and so the process begins again. Eventually the process will converge on the optimal policy, which also means we'll have found the optimal value function.

# Policy Iteration
Generalised Policy Iteration algorithms differ in how they interleave the evaluation and improvement steps. In Policy Iteration it waits for each step to complete before starting the next one. So, at each iteration, it will only update the policy after Policy Evaluation has converged.

We can test Policy Evaluation by expanding on our simple level, adding in a few more states, puddles and walls to increase the challenge. This new level is shown below:

In [9]:
level_width = 6
level_height = 3
level = GridLevel( level_width,level_height,start=[0,0])
robot_position = RobotPosition(level,initial_sprite = 2)

splashes = [[0,2,1,1,2,0],
            [2,1,1,2,1,2],
            [0,1,2,2,0,0]]
level.add_splashes( splashes )

# add some walls
walls = [((0, 0),'E'),((4, 2),'E'),((2, 2),'E'),((3, 1),'E'),((1, 1),'E'),((2, 0),'E')]
level.add_walls( walls )

level.canvases

MultiCanvas(height=196, sync_image_data=True, width=388)

<center><i>Figure 8: An expanded level to test Policy Iteration.</i></center>

Additionally, we'll switch from a stochastic policy to use a deterministic one. In this policy, shown in Figure 9, a single action is specified for each state.

In [10]:
level = GridLevel( level_width,level_height,start=[0,0])
robot_position = RobotPosition(level,initial_sprite = 2)
level.add_splashes( splashes )
level.add_walls( walls )

directions = np.array([[4, 2, 4, 4, 8, 8],
                       [4, 8, 1, 8, 4, 1],
                       [2, 1, 8, 2, 1, 0]])

policy = Policy(level)
policy.set_policy(directions)                       

level.show_directions(directions)
level.show_cell_direction_text(directions) 
level.canvases

MultiCanvas(height=196, sync_image_data=True, width=388)

<center><i>Figure 9: An initial deterministic policy to test Policy Iteration.</i></center>
  
Since some of the actions result in looped states, and there is no path that leads to the exit, policy evaluation will never converge. Therefore, to run policy evaluation, we either need to use discounted rewards or limit the evaluation step to a fixed number of iterations. 

Choosing the discount factor approach, and applying a value of 0.9, policy evaluation converges in 75 iterations. With these generated state values we can then act greedily and apply policy improvement to generate a new policy. This gives the state values and new policy shown in Figure 10 below.  

In [11]:
level = GridLevel( level_width,level_height,start=[0,0])
level.add_splashes( splashes )
level.add_walls( walls )

directions = np.array([[4, 2, 4, 4, 8, 8],
                       [4, 8, 1, 8, 4, 1],
                       [2, 1, 8, 2, 1, 0]])

# initially policy evaluation starts with all estimates set to zero
policy_evaluation = PolicyEvaluation( level )
policy_evaluation.set_policy(directions)
policy_evaluation.set_discount_factor(0.9)

# run until the change in estimated state values drops below the defined threshold
iterations = policy_evaluation.run_to_convergence(max_iterations = 300)
level.show_directions(directions)
level.show_values(policy_evaluation.end_values)
print(f"Convergence in {iterations} iterations")
level.canvases

Convergence in 75 iterations


MultiCanvas(height=196, sync_image_data=True, width=388)

<center><i>Figure 10: State values and new policy after the first run of Policy Iteration.</i></center>

After the first run of Policy Iteration, the policy is better than the initial policy and now has the rightmost states leading to the exit. However, there's still no path all the way from the start state to the exit. So we need to run another pass of Policy Iteration. The converged state values and new, improved, policy for this are shown below:

In [12]:
level = GridLevel( level_width,level_height,start=[0,0])
level.add_splashes( splashes )
level.add_walls( walls )

directions = np.array([[4, 2, 4, 4, 8, 8],
                       [4, 8, 1, 8, 4, 1],
                       [2, 1, 8, 2, 1, 0]])

policy = Policy(level)

# initially policy evaluation starts with all estimates set to zero
policy_evaluation = PolicyEvaluation( level )
policy_evaluation.set_policy(directions)
policy_evaluation.set_discount_factor(0.9)


iterations = policy_evaluation.run_to_convergence(max_iterations = 300)
directions = policy.update_policy(policy_evaluation.end_values)

# evaluate the new policy 
# - note its starting values are the state values from the previous policy
policy_evaluation.set_policy(directions)
level.show_values(policy_evaluation.start_values)

# run until the change in estimated state values drops below the defined threshold
iterations = policy_evaluation.run_to_convergence(max_iterations = 300)

# show the best actions after policy evaluation has converged
directions = policy.update_policy(policy_evaluation.end_values)
level.show_directions(directions)
level.show_values(policy_evaluation.end_values)

print(f"Convergence in {iterations} iterations")

level.canvases

Convergence in 55 iterations


MultiCanvas(height=196, sync_image_data=True, width=388)

<center><i>Figure 11: State values and new policy after the second run of Policy Iteration.</i></center>

The policy after the second step of Policy Iteration is slightly better than the last, but still hasn't found the optimal policy. For this we need to run for a few more steps. These are shown below, along with the number of iterations required to reach convergence (the 'Conv' value shown on the right) on each run of Policy Evaluation. It can be seen that the optimal policy is reached after 5 steps of Policy Iteration.

In [13]:
level = GridLevel( level_width,level_height,start=[0,0],show_end_text=False,side_panel=True)
level.add_splashes( splashes )
level.add_walls( walls )

# # this set of directions look like a good example policy
directions = np.array([[4, 2, 4, 4, 8, 8],
                       [4, 8, 1, 8, 4, 1],
                       [2, 1, 8, 2, 1, 0]])

# initially policy evaluation starts with all estimates set to zero
policy_evaluation = PolicyEvaluation( level )
policy_evaluation.set_policy(directions)
policy_evaluation.set_discount_factor(0.9)

policy = Policy(level)
policy.set_policy(directions)                       

level.show_directions(directions)
level.show_values(policy_evaluation.end_values)
level.side_panel_text(400,32,f"Step: {step}")
level.side_panel_text(400,64,f"Conv: 0")

step = 0
new_policy = policy.get_policy()

def on_update(*args):   
  global new_policy, step
  
  # note that policy evaluation starts with the state values from the previous policy
  # - this greatly reduces the number of iterations required to run policy evaluation to convergence
  
  # reset policy evaluation start values back to zero to see how the number of iterations is affected
#   policy_evaluation.reset()

  # run until the change in estimated state values drops below the defined threshold 
  policy_evaluation.set_policy(new_policy)
  iterations = policy_evaluation.run_to_convergence(max_iterations = 300)  
  level.side_panel_text(400,64,f"Conv: {iterations}")

  # show the best actions after policy evaluation has converged
  directions = policy.update_policy(policy_evaluation.end_values)
  level.show_directions(directions)
  level.show_values(policy_evaluation.end_values)  
  new_policy = policy.get_policy()
  step += 1
  level.side_panel_text( 400, 32, f"Step: {step}" ) 

NameError: name 'step' is not defined

In [ ]:
play, progress, layout = setup_play_level( level, on_update, max=7 )
VBox((level.canvases, HBox((play, progress))),layout=layout) 

<center><i>Figure 12: Running Policy Iteration until the optimal policy is discovered. 'Conv' is the number of iterations, required at each step, for Policy Evaluation to converge.</i></center>

In the diagram above, note how the number of iterations of policy evaluation required to reach convergence decreases after each Policy Iteration step. This is due to retaining the previous policy's converged state values when starting the next policy evaluation. Since the new policy is based on the previous policy, it will have similar state values. Therefore, using the previous policy's values greatly reduces the number of iterations required to run policy evaluation on the new policy.

<blockquote>
<i>
Policy Iteration takes an initial policy, evaluates it, and then uses those values to create an improved policy. These steps of evaluation and improvement are then repeated on the newly generated policy to give an even better policy. This process continues until, eventually, we end up with the optimal policy.
</i>
</blockquote>

# Value Iteration
As we've seen, Policy Iteration evaluates a policy and then uses these values to improve that policy. This process is repeated until eventually the optimal policy is reached. As a result, at each iteration prior to the optimal policy, a sub-optimal policy has to be fully evaluated. Consequently, there is potentially a lot of wasted effort when trying to find the optimal policy.

Look again at Policy Evaluation on the very simple grid level, shown in figure 13 below. Note how the blue arrows change at each iteration. These show a potential policy, formed from acting greedily with respect to the state values that are available at the current iteration.

In [ ]:
from value_iteration import ValueIteration

level_width = 3
level_height = 3
level = GridLevel( level_width,level_height,start=[0,0],end=[2,2],side_panel=True)

splashes = [[0,2,1],[0,2,1],[0,1,0]]
level.add_splashes( splashes )

walls = [((0, 0),'E'),((1, 2),'E')]
level.add_walls( walls )

In [ ]:
policy_evaluation = PolicyEvaluation( level )
policy = Policy(level)
directions = policy.get_directions(policy_evaluation.end_values)
step = 0

level.show_directions(directions)
level.show_values(policy_evaluation.end_values)
level.side_panel_text(210,28,f"Iteration: {step}")

def on_update(*args):   
  global step     
  policy_evaluation.do_iteration()
  directions = policy.get_directions(policy_evaluation.end_values)
  level.show_directions(directions)
  level.show_values(policy_evaluation.end_values)    
  level.side_panel_text(210,28,f"Iteration: {policy_evaluation.get_iterations()}")
  step += 1

In [ ]:
play, progress, layout = setup_play_level( level, on_update, max=62 )
VBox((level.canvases, HBox((play, progress))),layout=layout) 

<center><i>Figure 13: Policy Evaluation, showing how the policy would be updated if we acted greedily at each iteration.</i></center>

Previously we ran policy evaluation for this level to full convergence, taking 234 iterations, and then acted greedily, with respect to the converged state values, to get the optimal policy. However, if you look at how the policy would be updated if we instead acted greedily at each iteration, you can see that the optimal policy has actually been found after only 14 iterations. We didn't need to run evaluation to full convergence. The number of state value calculations could have been reduced dramatically and we would still have obtained the best policy.

Many GPI algorithms use this idea to go beyond simple Policy Iteration and instead improve their policy after a reduced number of Policy Evaluation steps. Value Iteration takes this idea to the extreme, effectively reducing the evaluation stage down to a single sweep of the states. Additionally, to improve things further, it combines the Policy Evaluation and Policy Improvement stages into a single update.

This can be seen in equation 3 below:

<br/>
<center><img src="images/part3/value_iteration.png"/></center>
<br/>
<center><i>Equation 3: Value Iteration. The value of state 's' at iteration 'k+1' is the value of the action that gives the maximum expected reward.</i></center>
<br/>


At iteration 'k+1' of Value Iteration, the value of state 's' is given by the value of the action, in that state, that returns the maximum expected reward. 

The expected value for an action is given by the sum over the transition probabilities for that action, multiplied by the reward obtained for making that transition, combined with the value of the state where you end up. Therefore the equation can be re-written as:


<br/>
<center><img src="images/part3/value_iteration_update.png"/></center>
<br/>
<center><i>Equation 4: Value Iteration. The value of state 's' at iteration 'k+1' is the value of the action that gives the maximum value. An action's value is the sum over the transition probabilities times the reward obtained for the transition combined with the discounted value of the next state.</i></center>
<br/>


In a state 's', we calculate the value of each action by taking the sum over the transition probabilities for that action, multiplied by the immediate reward summed with the discounted reward of the next state s′, using that state's current value. We then choose the largest of these calculated values and use this as the value of state 's' at the next iteration 'k+1'.

If you look back at Equation 2, where the Bellman expectation equation was expressed as an update function, you'll see that these equations are practically identical. The only difference is, in the original Policy Evaluation equation, the next state value was given by the sum over the policy's probability of taking each action, whereas now, in the Value Iteration equation, we simply take the value of the action that returns the largest value.

The values we calculated previously, when doing Policy Evaluation for the center square of the simple grid level, are shown below:

<br/>
<center><img src="images/part3/v_nsew_vi.png"/></center>
<br/>

When evaluating a stochastic policy, where each action was equally likely, we simply averaged over these 4 values to get the value of the estimated state value at the first iteration. With Value Iteration we instead take the maximum (least negative) value, which in this case is -2.0. Doing this for all states gives the first set of Value Iteration estimates shown below:

In [ ]:
level_width = 3
level_height = 3
level = GridLevel( level_width,level_height,start=[0,0],end=[2,2])

splashes = [[0,2,1],[0,2,1],[0,1,0]]
level.add_splashes( splashes )

walls = [((0, 0),'E'),((1, 2),'E')]
level.add_walls( walls )

value_iteration = ValueIteration( level,discount_factor=1.0 )
value_iteration.state_sweep()
level.show_values(value_iteration.values)

level.canvases

<center><i>Figure 14: The state values under Value Iteration after the first sweep.</i></center>

We now continue, just as we did for Policy Evaluation, to use our calculated values from the current iteration, to calculate the state values at the next iteration and run this to convergence. The state value updates at each iteration are shown below:

In [ ]:
level_width = 3
level_height = 3
level = GridLevel( level_width,level_height,start=[0,0],end=[2,2],side_panel=True)

splashes = [[0,2,1],[0,2,1],[0,1,0]]
level.add_splashes( splashes )

walls = [((0, 0),'E'),((1, 2),'E')]
level.add_walls( walls )

value_iteration = ValueIteration( level,discount_factor=1.0 )
level.show_values(value_iteration.values)

In [ ]:
step = 0
level.side_panel_text(210,28,f"Iteration: {step}")

def on_update(*args):   
  global step     
  step += 1
  value_iteration.state_sweep()    

  if step >= 50:
    # now select greedily wrt the converged values
    policy = Policy(level)
    directions = policy.get_directions(value_iteration.values)
    level.show_directions(directions)
  
  level.show_values(value_iteration.values)  
  level.side_panel_text(210,28,f"Iteration: {step}")

In [ ]:
play, progress, layout = setup_play_level( level, on_update, max=62 )
VBox((level.canvases, HBox((play, progress))),layout=layout) 

<center><i>Figure 15: Value Iteration run to convergence (convergence to 2 decimal places actually occurs in 62 iterations)</i></center>

Once the estimated state values have stopped changing, and convergence has been reached, we can select greedily with respect to these values to get the optimal policy. In the case of this simple level, convergence occurs and the optimal policy is found after 62 iterations.

So, where Policy Iteration would evaluate a policy and then act greedily with respect to those calculated values to get the next policy, Value Iteration effectively works without a policy. Only at the end, once the optimal value function has been found, will the optimal policy be created from the state values.

For this simple grid level it takes only 62 iterations for the state values to converge and for the optimal policy to be discovered. When you compare this against the 234 iterations that are required to run Policy Evaluation to convergence it can be seen how Value Iteration is a vast improvement over simple Policy Iteration. 

Additionally, since each iteration represents a sweep over all of the states, the reduction in the total number of iterations required to find the optimal policy will be of even greater benefit in more complex environments, featuring much larger state spaces. 

As an example, take a look at Value Iteration applied to the expanded grid level that we previously used to test Policy Iteration:

In [ ]:
level_width = 6
level_height = 3
level = GridLevel( level_width,level_height,start=[0,0],side_panel=True,show_end_text=False)

splashes = [[0,2,1,1,2,0],
            [2,1,1,2,1,2],
            [0,1,2,2,0,0]]
level.add_splashes( splashes )

# add some walls
walls = [((0, 0),'E'),((4, 2),'E'),((2, 2),'E'),((3, 1),'E'),((1, 1),'E'),((2, 0),'E')]
level.add_walls( walls )

value_iteration = ValueIteration( level,discount_factor=1.0 )
level.show_values(value_iteration.values)

In [ ]:
step = 0
policy = Policy(level)
level.side_panel_text(400,28,f"Iteration: {step}")

def on_update(*args):   
  global step     
  step += 1
  value_iteration.state_sweep()    

  directions = policy.get_directions(value_iteration.values)
  level.show_directions(directions) 
  level.show_values(value_iteration.values)  
  level.side_panel_text(400,28,f"Iteration: {step}")

In [ ]:
play, progress, layout = setup_play_level( level, on_update, max=30 )
VBox((level.canvases, HBox((play, progress))),layout=layout) 

<center><i>Figure 16: The first 30 iterations of Value Iteration for the expanded grid level.</i></center>

For this level the optimal policy is found in 25 iterations, although it takes a total of 120 iterations before the state values reach convergence. If you compare this to Policy Iteration, where it took a total of 203 iterations to find the optimal state values, you can see the large improvement in performance.

The optimal policy and final, converged, state values for this level are shown in Figure 17 below. Since we didn't use a discount factor when running Value Iteration, for each state, these values represent the average number of time steps it should take for Baby Robot to reach the exit from that state.

In [ ]:
level = GridLevel( level_width,level_height,start=[0,0])
level.add_splashes( splashes )
level.add_walls( walls )

value_iteration = ValueIteration( level,discount_factor=1.0 )
level.show_values(value_iteration.values)

In [ ]:
value_iteration = ValueIteration( level,discount_factor=1.0 )
level.show_values(value_iteration.values)
iterations = value_iteration.run_to_convergence(max_iterations=200)

# now select greedily wrt the converged values
policy = Policy(level)
directions = policy.get_directions(value_iteration.values)
level.show_directions(directions)
level.show_values(value_iteration.values)

print(f"Values converged in {iterations} iterations")
level.canvases

<center><i>Figure 17: The state values after Policy Iteration has converged in 120 iterations.</i></center>

---

# Summary
With Policy Evaluation we were already able to evaluate the states in a Reinforcement Learning environment. By adding Policy Improvement we can then take these values and form a new, improved, policy. 

This combination of Policy Evaluation and Policy Improvement is known as <b>Generalised Policy Iteration (GPI)</b> and, through it's application, an initial policy can be repeatedly evaluated and improved until the optimal policy is found.

In <b>Policy Iteration</b>, at each step, policy evaluation is run until convergence, then the policy is updated and the process repeats.

In contrast, <b>Value Iteration</b> only does a single iteration of policy evaluation at each step. Then, for each state, it takes the maximum action value to be the estimated state value. Once these state values have converged, to the optimal state values, then the optimal policy can be obtained. In practice this performs much better than Policy Iteration and finds the optimal state value function in much fewer steps.

---

# What's Next?
In an environment where we know exactly the probabilities of moving from one state to the next, and the rewards that will be obtained for each of these transitions, we can apply Dynamic Programming to evaluate each state in the environment, and from this we can find the optimal policy. 

However, these methods don't actually learn anything. The complete model of the environment is given and so it's not necessary to perform any exploration to form this model. 

In the next part we'll look at Monte Carlo methods, which don't require complete knowledge of the environment and instead learn from experience.

![](images/green_babyrobot_small.gif)